# Impact of the backend modeling on the reconstructed mass

This notebook was setup to guide the discussion in section 3.2.3 of the CLMM v1.0 paper.

In [ ]:
import os

os.environ["CLMM_MODELING_BACKEND"] = "ct"
import sys
import math
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# CLMM with ct be
import clmm
import clmm.theory as mod
from clmm import Cosmology as clmm_cosmo
from clmm.support.sampler import fitters
from clmm.constants import Constants as const

In [ ]:
import cluster_toolkit as ct

In [ ]:
# NumCosmo
try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

In [ ]:
# Colossus
from colossus.cosmology import cosmology as col_cosmo

In [ ]:
# CCL
import pyccl as ccl

## Define comology and prepare backends

In [ ]:
# Colossus Cosmology
cosmo_col = col_cosmo.setCosmology("planck18")
# no massive neutrinos

Omega_c_col = cosmo_col.Om0 - cosmo_col.Ob0
Omega_b_col = cosmo_col.Ob0
h_col = cosmo_col.H0 / 100.0
n_s_col = cosmo_col.ns

# CCL Cosmology
cosmo_ccl = ccl.Cosmology(
    Omega_c=Omega_c_col, Omega_b=Omega_b_col, h=h_col, A_s=2.1e-9, n_s=n_s_col
)

# NumCosmo Cosmology
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

cosmo = Nc.HICosmo.new_from_name(Nc.HICosmo, "NcHICosmoDEXcdm{'massnu-length':<0>}")
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", cosmo_col.H0)
cosmo.param_set_by_name("Omegak", cosmo_col.Ok(0.0))
cosmo.param_set_by_name("w", cosmo_col.w0)
cosmo.param_set_by_name("Omegab", Omega_b_col)
cosmo.param_set_by_name("Omegac", Omega_c_col)
cosmo.param_set_by_name("ENnu", cosmo_col.Neff)
cosmo.param_set_by_name("Tgamma0", cosmo_col.Tcmb0)

# CLMM Cosmology: with CT backend, massive neutrinos not taken into account
cosmo_clmm = clmm_cosmo(H0=cosmo_col.H0, Omega_dm0=Omega_c_col, Omega_b0=Omega_b_col, Omega_k0=0.0)
print(cosmo_col)

In [ ]:
Delta = 200
Mvir = 1.0e15  # M_sun
cvir = 5.0
z = 0.2
a = 1.0 / (1.0 + z)

# CCL
MDEF = "matter"
mdef = ccl.halos.MassDef(Delta, MDEF)
conc = ccl.halos.ConcentrationConstant(cvir)
mdef.concentration = conc
ccl_nfw_num = ccl.halos.HaloProfileNFW(
    conc, truncated=False, projected_analytic=False, cumul2d_analytic=False
)
ccl_nfw_ana = ccl.halos.HaloProfileNFW(
    conc, truncated=False, projected_analytic=True, cumul2d_analytic=True
)
# ccl_nfw_num.update_precision_fftlog (n_per_decade = 1200)
ccl_ein = ccl.halos.HaloProfileEinasto(conc, truncated=False)
ccl_her = ccl.halos.HaloProfileHernquist(conc, truncated=False)


alpha = ccl_ein._get_alpha(cosmo_ccl, Mvir, a, mdef)

In [ ]:
# NumCosmo
reltol = 1.0e-9
lnXi = math.log(1.0e-4)
lnXf = math.log(1.0e4)
improve_prec = False

nc_nfw = Nc.HaloDensityProfileNFW.new(Nc.HaloDensityProfileMassDef.MEAN, Delta)
nc_nfw.props.cDelta = cvir
nc_nfw.props.log10MDelta = math.log10(Mvir)

if improve_prec:
    nc_nfw.set_reltol(reltol)
    nc_nfw.set_lnXi(lnXi)
    nc_nfw.set_lnXf(lnXf)

nc_ein = Nc.HaloDensityProfileEinasto.new(Nc.HaloDensityProfileMassDef.MEAN, Delta)
nc_ein.props.cDelta = cvir
nc_ein.props.log10MDelta = math.log10(Mvir)
nc_ein.props.alpha = alpha

if improve_prec:
    nc_ein.set_reltol(reltol)
    nc_ein.set_lnXi(lnXi)
    nc_ein.set_lnXf(lnXf)

nc_her = Nc.HaloDensityProfileHernquist.new(Nc.HaloDensityProfileMassDef.MEAN, Delta)
nc_her.props.cDelta = cvir
nc_her.props.log10MDelta = math.log10(Mvir)

if improve_prec:
    nc_her.set_reltol(reltol)
    nc_her.set_lnXi(lnXi)
    nc_her.set_lnXf(lnXf)

smd = Nc.WLSurfaceMassDensity.new(Nc.Distance.new(5.0))
smd.prepare(cosmo)

## Prepare an ideal data vector
- Using analytical CCL for NFW
- Using NumCosmo for Einasto

In [ ]:
r = np.logspace(-3, 1, 100)  # project radius in Mpc

In [ ]:
# NumCosmo
nc_DeltaSigma_ein = np.array(smd.sigma_excess_array(nc_ein, cosmo, r, 1.0, 1.0, z))


# CCL
ccl_Sigma_nfw_ana = ccl_nfw_ana.projected(cosmo_ccl, r / a, Mvir, a, mdef) / a**2
ccl_BarSigma_nfw_ana = ccl_nfw_ana.cumul2d(cosmo_ccl, r / a, Mvir, a, mdef) / a**2
ccl_DeltaSigma_nfw_ana = ccl_BarSigma_nfw_ana - ccl_Sigma_nfw_ana

## Naive test
- Add a constant bias to the model over the full radial range and fit the mass using ideal $\Delta\Sigma$ data generated above using the unbiased model
- Repeat for NFW and Einasto
- plot the bias on the reconstructed mass as a function of the input bias

In [ ]:
def model_ref_nfw(r, logm, bias):
    m = 10.0**logm
    tmp_Sigma = ccl_nfw_ana.projected(cosmo_ccl, r / a, m, a, mdef) / a**2
    tmp_BarSigma = ccl_nfw_ana.cumul2d(cosmo_ccl, r / a, m, a, mdef) / a**2
    model = (tmp_BarSigma - tmp_Sigma) * (1 + bias)
    return model


def model_ref_ein(r, logm, bias):
    m = 10.0**logm
    nc_ein.props.log10MDelta = logm
    return np.array(smd.sigma_excess_array(nc_ein, cosmo, r, 1.0, 1.0, z)) * (1 + bias)

In [ ]:
logmass_nfw = []
logmass_ein = []
bias = np.logspace(-6, np.log10(0.5), 30)
for b in bias:
    popt1, pcov1 = fitters["curve_fit"](
        lambda rr, logm: model_ref_nfw(rr, logm, b),
        r,
        ccl_DeltaSigma_nfw_ana,
        np.zeros(len(r)) + 1.0e-30,
        bounds=[13.0, 17.0],
    )
    popt2, pcov2 = fitters["curve_fit"](
        lambda rr, logm: model_ref_ein(rr, logm, b),
        r,
        nc_DeltaSigma_ein,
        np.zeros(len(r)) + 1.0e-30,
        bounds=[13.0, 17.0],
    )

    logmass_nfw.append(popt1[0])
    logmass_ein.append(popt2[0])

In [ ]:
reldiff_nfw = np.abs(10 ** np.array(logmass_nfw) - Mvir) * 100.0 / Mvir
reldiff_ein = np.abs(10 ** np.array(logmass_ein) - Mvir) * 100.0 / Mvir
plt.scatter((bias) * 100, reldiff_nfw, color="orange")
plt.scatter((bias) * 100, reldiff_ein, color="blue", marker="+")
plt.plot(bias * 100, 2.4 * bias * 100)  # "Fit by eye"
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"Hypothetical bias on $\Delta\Sigma$ [%]")
plt.ylabel(r"Bias on reconstructed mass [%]")

## More realistic test for NFW
- Use the same set of ideal NFW data points, generated from the analytical CCL prescription
- Adjust the mass using the analytical and numerical NFW implementation of all backends
- Print the mass bias

### Define all the models

In [ ]:
# CCL numerical
def model_num_ccl(r, logm):
    m = 10.0**logm
    tmp_Sigma = ccl_nfw_num.projected(cosmo_ccl, r / a, m, a, mdef) / a**2
    tmp_BarSigma = ccl_nfw_num.cumul2d(cosmo_ccl, r / a, m, a, mdef) / a**2
    model = tmp_BarSigma - tmp_Sigma
    return model

In [ ]:
# CT analytical
def model_ana_ct(r, logm):
    m = 10.0**logm
    return mod.compute_excess_surface_density(
        r, m, cvir, z, cosmo=cosmo_clmm, delta_mdef=Delta, halo_profile_model="nfw"
    )

In [ ]:
# CT numerical
def model_num_ct(r, logm):
    cor_factor = clmm.utils._patch_rho_crit_to_cd2018(2.77533742639e11)
    omega_m = cosmo_clmm.get_E2Omega_m(z) * cor_factor

    rhocrit_mks = 3.0 * 100.0 * 100.0 / (8.0 * np.pi * const.GNEWT.value)
    rhocrit_cosmo = (
        rhocrit_mks * 1000.0 * 1000.0 * const.PC_TO_METER.value * 1.0e6 / const.SOLAR_MASS.value
    )

    r_for_sigma = np.logspace(-6, 2, len(r) * 1000)
    r3d = np.logspace(-7, 3, len(r) * 1000)

    m = 10.0**logm

    rho_nfw = ct.density.rho_nfw_at_r(
        r3d * cosmo_clmm["h"], m * cosmo_clmm["h"], cvir, omega_m, delta=Delta
    )
    integrand_nfw = rho_nfw / (omega_m * rhocrit_cosmo)  # xi_nfw + 1 (see CT documentation)

    # Redefine Sigma on finer grid for the numerical integration for DeltaSigma
    tmp_nfw = (
        ct.deltasigma.Sigma_at_R(
            r_for_sigma * cosmo_clmm["h"],
            r3d * cosmo_clmm["h"],
            integrand_nfw,
            m * cosmo_clmm["h"],
            cvir,
            omega_m,
        )
        * cosmo_clmm["h"]
        * 1.0e12
    )  # Msun/Mpc2
    return ct.deltasigma.DeltaSigma_at_R(
        r * cosmo_clmm["h"],
        r_for_sigma * cosmo_clmm["h"],
        tmp_nfw,
        m * cosmo_clmm["h"],
        cvir,
        omega_m,
    )

In [ ]:
# NC analytical
def model_ana_numcosmo(r, logm):
    nc_nfw.props.log10MDelta = logm
    return np.array(smd.sigma_excess_array(nc_nfw, cosmo, r, 1.0, 1.0, z))

In [ ]:
# NC numerical
def model_num_numcosmo(r, logm):
    nc_nfw.props.log10MDelta = logm
    Nc.halo_density_profile_nfw_class_set_ni(True)
    res = np.array(smd.sigma_excess_array(nc_nfw, cosmo, r, 1.0, 1.0, z))
    Nc.halo_density_profile_nfw_class_set_ni(False)
    return res

### Perform the adjustment using `curve_fit`
- Negligible errors (1.e-15) are added for curve_fit not to crash.

In [ ]:
# CCL num
popt1, pcov1 = fitters["curve_fit"](
    lambda rr, logm: model_num_ccl(rr, logm),
    r,
    ccl_DeltaSigma_nfw_ana,
    np.zeros(len(r)) + 1.0e-15,
    bounds=[13.0, 17.0],
)
print(popt1, pcov1)
print(f"Mass bias for CCL numerical = {np.abs(10**np.array(popt1[0]) - Mvir)*100./Mvir} %")

In [ ]:
# NC analytical
popt1, pcov1 = fitters["curve_fit"](
    lambda rr, logm: model_ana_numcosmo(rr, logm),
    r,
    ccl_DeltaSigma_nfw_ana,
    np.zeros(len(r)) + 1.0e-15,
    bounds=[13.0, 17.0],
)
print(popt1, pcov1)
print(f"Mass bias for NC analytical = {np.abs(10**np.array(popt1[0]) - Mvir)*100./Mvir} %")

In [ ]:
# NC numerical
popt1, pcov1 = fitters["curve_fit"](
    lambda rr, logm: model_num_numcosmo(rr, logm),
    r,
    ccl_DeltaSigma_nfw_ana,
    np.zeros(len(r)) + 1.0e-15,
    bounds=[13.0, 17.0],
)
print(popt1, pcov1)
print(f"Mass bias for NC numerical = {np.abs(10**np.array(popt1[0]) - Mvir)*100./Mvir} %")

In [ ]:
# CT analytical
popt1, pcov1 = fitters["curve_fit"](
    lambda rr, logm: model_ana_ct(rr, logm),
    r,
    ccl_DeltaSigma_nfw_ana,
    np.zeros(len(r)) + 1.0e-15,
    bounds=[10.0, 17.0],
)
print(popt1, pcov1)
print(f"Mass bias for CT analytical = {np.abs(10**np.array(popt1[0]) - Mvir)*100./Mvir} %")

In [ ]:
# CT numerical
popt1, pcov1 = fitters["curve_fit"](
    lambda rr, logm: model_num_ct(rr, logm),
    r,
    ccl_DeltaSigma_nfw_ana,
    np.zeros(len(r)) + 1.0e-15,
    bounds=[10.0, 17.0],
)
print(popt1, pcov1)
print(f"Mass bias for CT numerical = {np.abs(10**np.array(popt1[0]) - Mvir)*100./Mvir} %")